In [1]:
from data4ml import Data4ML
from bs4 import BeautifulSoup
import json
import os
import re
from perfect_regex import (
    perfect_url_regex,
    perfect_emoji_regex,
    perfect_email_regex,
    perfect_phone_regex,
)

In [117]:
class Data4Chatbot(Data4ML):
    
    def __init__(self, max_length=10, is_normalize=True, **kwargs):
        super().__init__(**kwargs)
        self.is_normalize = is_normalize
        self.max_length = max_length
        
    def make_data(self):
        pass
    
    def parse_html(self, parent_folder: str, files: list) -> list:

        assert isinstance(files, list)
        all_messages = []
        for file in files:
            with open(os.path.join(parent_folder, file), "rb") as f:
                soup = BeautifulSoup(f, "lxml")

            messages = []
            for message in soup.find_all("div", {"class": "message"}):
                message = message.text.strip()
                messages.append(message)

            # Reverse the list to save the message sequence:
            all_messages.extend(messages[::-1])
        return all_messages
    
    def normalize_message(self, s: str) -> str:
        """Lowercase, trim, and remove non-letter characters"""
        
        s = s.lower().strip()
        
        # Add space before `.`, `!` or `?`:
        s = re.sub(r"([.!?])", r" \1", s)
        
        # Remove non-letter characters:
        s = re.sub(r"[^а-яА-ЯёЁa-zA-Z.!?]+", r" ", s)
        s = re.sub(r"\s+", r" ", s).strip()
        return s 
    
    
    def transform_message(self, s: str) -> str:
        """Make first char upper and add a dot in the end of sentence."""
        
        if s:
            s = s[0].upper() + s[1:]
            if s[-1].isalpha():
                s += '.'
        if self.is_normalize:
            s = self.normalize_message(s)
        return s

    def filter_pair(self, p):
        """Returns True if both sentences in a pair 'p' are under the MAX_LENGTH threshold"""
        # Input sequences need to preserve the last word for EOS token
        return len(p[0].split(' ')) < self.max_length and len(p[1].split(' ')) < self.max_length

    def filter_pairs(self, pairs):
        """Filter pairs using filterPair condition"""
        return [pair for pair in pairs if self.filter_pair(pair)]
    
    def get_pairs(self, messages: list) -> list:
        pairs = [[messages[i-1], messages[i]] for i in range(1, len(messages))]
        return self.filter_pairs(pairs)
    
    def clear_messages(self, messages: list) -> list:
        messages = []
        
        
        # Who start the dialog:
        last_author = all_messages[0][:all_messages[0].find(',')]
        skip_next_sent = False
        for message in all_messages:
            author =  message[:message.find(',')]
            message = message[message.find("\n") + 1 :]
            clear_message = self.transform_message(chatbot._clear_message(message))
            if clear_message:
                    
                if author == last_author: # still one message
                    if messages:
                        messages[-1] += (' ' + clear_message)
                    else: # for the first iteration
                        messages.append(clear_message)
                else: # if author change
                    messages.append(clear_message)
                    
                last_author = author 
            
            #TODO: 
            """
            Продумать насчёт пустых сообщений. Например, это сообщение:
                Вы: Привет, скинь фотку.
                Они: [Фото#1]
                Вы: Спасибо
                Они: [Фото#2]
                Вы: Большое спасибо!
                
            Очистится в:
                Вы: Привет, скинь фотку.
                Вы: Спасибо
                Вы: Большое спасибо!
                
            Если менять автора каждый раз, то получится как выше, а если не менять, то::
                Вы: Привет, скинь фотку. Спасибо. Большое спасибо!
                
            И что вот лучше?
            """ 
        return messages

In [118]:
chatbot = Data4Chatbot(path_to_config='./../data_params.json')

In [119]:
list_of_folders = chatbot.get_list_of_folders('data4test')
print(list_of_folders)

['153164713', '153164714']


In [120]:
list_of_files = chatbot.get_list_of_files_in_folder('data4test/153164714')
print(list_of_files)

['messages0.html']


In [121]:
all_messages = chatbot.parse_html('data4test/153164714', list_of_files)
all_messages

['Юлия Николаева, 23 мар 2016 в 20:48:28\nпришли фотки, пожалуйста',
 'Юлия Николаева, 23 мар 2016 в 20:48:55\nwrong@yandex.ru',
 'Вы, 23 мар 2016 в 21:51:32\nПривет, сейчас',
 'Вы, 23 мар 2016 в 21:53:35\nВсё👌',
 'Юлия Николаева, 23 мар 2016 в 22:50:04\nСпасибо большое',
 'Вы, 23 мар 2016 в 22:53:36\n👌',
 'Юлия Николаева, 20 апр 2016 в 10:39:48\nСережка, перешли мне письмо Смоленцевой.\n1 прикреплённое сообщение',
 'Юлия Николаева, 20 апр 2016 в 10:40:12\nЗаранее спасибо',
 'Вы, 20 апр 2016 в 10:47:08\nКакое именно?',
 'Юлия Николаева, 20 апр 2016 в 10:47:36\nПоследнее. Про кино',
 'Юлия Николаева, 20 апр 2016 в 10:48:44\nСпасибо',
 'Вы, 20 апр 2016 в 10:48:44\nСкинул',
 'Вы, 20 апр 2016 в 22:16:23\nПодойдите завтра к смоленцевой',
 'Юлия Николаева, 20 апр 2016 в 22:20:43\nзачем?',
 'Вы, 21 апр 2016 в 5:26:20\nОна попросила',
 'Юлия Николаева, 21 апр 2016 в 8:37:28\nХорошо',
 'Вы, 22 апр 2016 в 18:36:12\nЮль',
 'Вы, 22 апр 2016 в 18:36:12\nне забывай, что вы завтра в 12 танцуете',
 'В

In [122]:
mes = chatbot.clear_messages(all_messages)
mes

['пришли фотки пожалуйста .',
 'привет сейчас . всё',
 'спасибо большое . сережка перешли мне письмо смоленцевой . заранее спасибо .',
 'какое именно ?',
 'последнее . про кино . спасибо .',
 'скинул . подойдите завтра к смоленцевой .',
 'зачем ?',
 'она попросила .',
 'хорошо .',
 'юль . не забывай что вы завтра в танцуете . и своим передай .',
 'хорошо . спасибо . спасибо . а до скольких ? спасибо .']

In [123]:
chatbot.get_pairs(mes)

[['пришли фотки пожалуйста .', 'привет сейчас . всё'],
 ['какое именно ?', 'последнее . про кино . спасибо .'],
 ['последнее . про кино . спасибо .',
  'скинул . подойдите завтра к смоленцевой .'],
 ['скинул . подойдите завтра к смоленцевой .', 'зачем ?'],
 ['зачем ?', 'она попросила .'],
 ['она попросила .', 'хорошо .']]

In [24]:

def transform_message(message: str) -> str:
    """Make first char upper and add a dot in the end of sentence."""
    # TODO: normilise text

    if message:
        message = message[0].upper() + message[1:]
        if message[-1].isalpha():
            message += '.'
    return message

messages = []
last_author = all_messages[0][:all_messages[0].find(',')] # Человек, начавший беседу

for message in all_messages:
    author =  message[:message.find(',')]
    message = message[message.find("\n") + 1 :]
    clear_message = chatbot._clear_message(message)
    if not clear_message:
        last_author =  author 
        print('lol')
        continue
        
    if author == last_author: # still one message
        if messages:
            messages[-1] += (' ' + transform_message(clear_message))
        else: # for the first iteration
            messages.append(transform_message(clear_message))
    else: # if author change
        messages.append(transform_message(clear_message))

    last_author =  author   

        

lol
lol
lol


In [25]:
messages

['Пришли фотки, пожалуйста.',
 'Привет, сейчас. Всё👌',
 'Спасибо большое.',
 '👌',
 'Сережка, перешли мне письмо Смоленцевой. Заранее спасибо.',
 'Какое именно?',
 'Последнее. Про кино. Спасибо.',
 'Скинул. Подойдите завтра к смоленцевой.',
 'Зачем?',
 'Она попросила.',
 'Хорошо.',
 'Юль. Не забывай, что вы завтра в 12 танцуете. И своим передай.',
 'Хорошо. Спасибо.',
 'Спасибо.',
 'А до скольких? Спасибо.']

In [33]:
messages

[':Юлия Николаева: пришли фотки, пожалуйста',
 ':Вы: Привет, сейчас',
 ':Вы: Всё👌',
 ':Юлия Николаева: Спасибо большое',
 ':Вы: 👌',
 ':Юлия Николаева: Сережка, перешли мне письмо Смоленцевой.',
 ':Юлия Николаева: Заранее спасибо',
 ':Вы: Какое именно?',
 ':Юлия Николаева: Последнее. Про кино',
 ':Юлия Николаева: Спасибо',
 ':Вы: Скинул',
 ':Вы: Подойдите завтра к смоленцевой',
 ':Юлия Николаева: зачем?',
 ':Вы: Она попросила',
 ':Юлия Николаева: Хорошо',
 ':Вы: Юль',
 ':Вы: не забывай, что вы завтра в 12 танцуете',
 ':Вы: И своим передай',
 ':Юлия Николаева: Хорошо',
 ':Юлия Николаева: спасибо',
 ':Юлия Николаева: Спасибо',
 ':Юлия Николаева: а до скольких?',
 ':Юлия Николаева: спасибо']

In [53]:
last_author = re.sub(': .*$', '', messages[0])[1:]
tmp = []

for i in messages:
    author = re.sub(': .*$', '', i)[1:]
    message = re.sub(':.*: ', '', i)
    
    if author == last_author: # значит это всё ещё сообщения от одного пользователя - tmp уже заполнен
        if tmp:
            tmp[-1] += (' ' + transform_message(message))
        else: # для первой итерации
            tmp.append(transform_message(message))
    else: # если пользователь сменился
        tmp.append(transform_message(message))
    
    last_author =  author   
        
tmp

['Пришли фотки, пожалуйста.',
 'Привет, сейчас. Всё👌',
 'Спасибо большое.',
 '👌',
 'Сережка, перешли мне письмо Смоленцевой. Заранее спасибо.',
 'Какое именно?',
 'Последнее. Про кино. Спасибо.',
 'Скинул. Подойдите завтра к смоленцевой.',
 'Зачем?',
 'Она попросила.',
 'Хорошо.',
 'Юль. Не забывай, что вы завтра в 12 танцуете. И своим передай.',
 'Хорошо. Спасибо. Спасибо. А до скольких? Спасибо.']

In [63]:
[[tmp[i-1], tmp[i]] for i in range(1, len(tmp))]

[['Пришли фотки, пожалуйста.', 'Привет, сейчас. Всё👌'],
 ['Привет, сейчас. Всё👌', 'Спасибо большое.'],
 ['Спасибо большое.', '👌'],
 ['👌', 'Сережка, перешли мне письмо Смоленцевой. Заранее спасибо.'],
 ['Сережка, перешли мне письмо Смоленцевой. Заранее спасибо.',
  'Какое именно?'],
 ['Какое именно?', 'Последнее. Про кино. Спасибо.'],
 ['Последнее. Про кино. Спасибо.', 'Скинул. Подойдите завтра к смоленцевой.'],
 ['Скинул. Подойдите завтра к смоленцевой.', 'Зачем?'],
 ['Зачем?', 'Она попросила.'],
 ['Она попросила.', 'Хорошо.'],
 ['Хорошо.', 'Юль. Не забывай, что вы завтра в 12 танцуете. И своим передай.'],
 ['Юль. Не забывай, что вы завтра в 12 танцуете. И своим передай.',
  'Хорошо. Спасибо. Спасибо. А до скольких? Спасибо.']]